### Build a very simple proof reading interface with Neuroglancer

In [1]:
import os
from urllib.parse import urlparse
import neuroglancer
from IPython.display import display, HTML

# Uncomment the following if you intend to run from a remote server:
# neuroglancer.set_server_bind_address('0.0.0.0')

neuroglancer.set_static_content_source(url="https://neuroglancer-demo.appspot.com/python")
viewer = neuroglancer.Viewer()

if "BINDER_URL" in os.environ:
    if "ovh.mybinder.org" in os.environ["BINDER_URL"]:
        binder_host = "hub-binder.mybinder.ovh"
    else:
        binder_host = "hub.gke.mybinder.org"
        
    viewer_url = "https://{0}{1}proxy/{2}{3}".format(
        binder_host,
        os.environ["JUPYTERHUB_SERVICE_PREFIX"],
        urlparse(viewer.get_viewer_url()).port,
        urlparse(viewer.get_viewer_url()).path)
else:
    viewer_url = viewer.get_viewer_url()

display(HTML('Neuroglancer link: <a href="{0}">{0}</a>'.format(viewer_url)))

In [2]:
# Create a 10x10x10 grid of unique labels
import numpy as np
dim = 20
data = np.arange(dim*dim*dim, dtype=np.uint64).reshape(dim, dim, dim)
# Make the labels 4x4x4 (for better meshes :-) [without scipy.ndimage.zoom]
data = np.repeat(np.repeat(np.repeat(data, 4, 0), 4, 1), 4, 2) 

In [3]:
# Create a new segmentation layer with our labels
with viewer.txn() as s:
        s.layers['labels'] = neuroglancer.SegmentationLayer(
            source = neuroglancer.LocalVolume(
                data=data, voxel_size=(1, 1, 1),
                mesh_options={'max_quadrics_error':-1}
            ),
            hideSegmentZero = False
    )

In [4]:
# Create an annotation layer to use for proof reading, linked to the label layer
with viewer.txn() as s:
    s.layers['proof'] = neuroglancer.AnnotationLayer(
        linkedSegmentationLayer="labels"
    )

In [5]:
# Try adding points to the proof layer:
# Select the layer with control-click on the name and then choose the "point" tool.
# Add points with control-click.

# Look at which points we have...
print(viewer.state.layers["proof"].annotations)

[{"point": [22.741104125976562, 27.827922821044922, 39.99999237060547], "type": "point", "id": "1052326476fa4033150ac59f5a2b4a1f546c69a2", "segments": ["3725"]}, {"point": [33.8231315612793, 40.36333084106445, 39.99999237060547], "type": "point", "id": "42df7b3dc49f45ba22f3cad24377e6c57e22b8db", "segments": ["3808"]}, {"point": [56.89554977416992, 62.3457145690918, 39.99999237060547], "type": "point", "id": "08d22a62da6bccddc5575ee72a0d5b0dd4977aec", "segments": ["3914"]}, {"point": [24.376157760620117, 65.97916412353516, 39.99999237060547], "type": "point", "id": "c28e9e9a43b0cf067bfc57f7ffcb458944530342", "segments": ["3926"]}]


In [6]:
# Bind a new 'merge' function to identify the chosen labels and create a segment-group
def merge_labels(s):
    segments = set()
    for segment in s.viewer_state.layers["proof"].annotations:
        if segment.segments:
            segments.update(segment.segments)
    with viewer.txn() as txn:
        for segment in segments:
            txn.layers["labels"].equivalences.union(max(segments), segment)
        txn.layers['proof'].annotations = None
    with viewer.config_state.txn() as txn:
        txn.status_messages['merge'] = 'Merged %s segments' % (len(segments))

# viewer.actions.clear()
viewer.actions.add('merge-labels', merge_labels)
with viewer.config_state.txn() as s:
    s.input_event_bindings.viewer['keym'] = 'merge-labels'